In [1]:
import numpy as np 
import pandas as pd 
import pymc3 as pm 
import theano.tensor as tt 

from bokeh.plotting import figure, show
from bokeh.models import BoxAnnotation, Span, Label, Legend
from bokeh.io import output_notebook
from bokeh.palettes import brewer
output_notebook()

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

Loading BokehJS ...

In [2]:
bd = pd.read_csv('./numpyro/birthdays.txt')

lam_date_join = lambda row: f"{row['year']}-{row['month']}-{row['day']}"
bd.index = pd.to_datetime(bd.apply(lam_date_join, axis=1), format="%Y-%m-%d")
bd = bd.drop(['year','month','day','day_of_year','day_of_week'],axis=1)
# bd = bd[bd.index < '1975-01-01']
bd

,births
1969-01-01,8486
1969-01-02,9002
1969-01-03,9542
1969-01-04,8960
1969-01-05,8390
...,...
1988-12-27,11528
1988-12-28,11847
1988-12-29,11704
1988-12-30,11837


In [3]:
def dates_to_idx(timelist):
    """
        Convert datetimes to numbers. From CO2 tutorial.
    """
    reference_time = pd.to_datetime('1969-01-01')
    t = (timelist - reference_time) / pd.Timedelta(365, "D")
    return np.asarray(t)

t = dates_to_idx(bd.index)

# normalize the birth rates
y = bd['births'].values
first_birth = y[0]
std_birth = np.std(y)
y_n = (y - first_birth)/std_birth

#
bd = bd.assign(t = t)
bd = bd.assign(y_n = y_n)

In [4]:
# train-test split
sep_idx = bd.index.searchsorted(pd.to_datetime('1985-01-01'))
data_early = bd.iloc[:sep_idx+1, :]
data_later = bd.iloc[sep_idx:, :]

In [5]:
p = figure(x_axis_type='datetime', title='Births Over Time',
           plot_width=550, plot_height=350)
p.yaxis.axis_label = 'Births'
p.xaxis.axis_label = 'Date'
predict_region = BoxAnnotation(left=pd.to_datetime("1985-01-01"),
                               fill_alpha=0.1, fill_color="firebrick")
p.add_layout(predict_region)
ppm400 = Span(location=400,
              dimension='width', line_color='red',
              line_dash='dashed', line_width=2)
p.add_layout(ppm400)

p.line(bd.index, bd['births'],
       line_width=2, line_color="black", alpha=0.5)
p.circle(bd.index, bd['births'],
         line_color="black", alpha=0.1, size=2)

train_label = Label(x=100, y=245, x_units='screen', y_units='screen',
                    text='Training Set', render_mode='css', border_line_alpha=0.0,
                    background_fill_alpha=0.0)
test_label  = Label(x=300, y=245, x_units='screen', y_units='screen',
                    text='Test Set', render_mode='css', border_line_alpha=0.0,
                    background_fill_alpha=0.0)

p.add_layout(train_label)
p.add_layout(test_label)
show(p)

We expect a weekly and a yearly periodic trend, as well as a long-term overall trend and a short-term trend. This will be a model with 5 kernels, one for each of the aforementioned processes, and one for observation noise:

$$ f\left(t\right) \sim GP_{slow}\left(0, k_1\left(t,t'\right)\right) + GP_{med}\left(0, k_2\left(t,t'\right)\right) + GP_{yearly}\left(0, k_3\left(t,t'\right)\right) + GP_{weekly}\left(0, k_4\left(t,t'\right)\right) + GP_{noise}\left(0, k_n\left(t,t'\right)\right)$$

* $GP_{slow}$ is a long-term, smoothly varying trend, exponential-quadratic kernel;
* $GP_{med}$ is a shorter-term, smoothly varying trend, rational-quadratic kernel; (maybe exp-quad?)
* $GP_{yearly}$ is periodic, yearly trend;
* $GP_{weekly}$ is periodic, weekly trend;
* $GP_{noise}$ is observation noise kernel.

To best design this model, we need to address the lengthscale, period, and scale parameter priors. The periods are known to us, and so their priors can be made tight, while the lengthscales priors may be less informative.

Note that most of the lengthscale priors are very uninformative. The medium-scale trend lengthscale prior is more informative biased toward smaller timescales. Similarly, the lengthscales for the periodic decays are also biased toward smaller timescales.

In [6]:
x = np.linspace(0, 400, 5000)
priors = [
    ("ℓ_pdecay_yearly",  pm.Gamma.dist(alpha=10, beta=0.075)), 
    ("ℓ_pdecay_weekly",  pm.Gamma.dist(alpha=10, beta=0.075)), 
    ("ℓ_psmooth_yearly", pm.Gamma.dist(alpha=4,  beta=3)),
    ("ℓ_psmooth_weekly", pm.Gamma.dist(alpha=4,  beta=3)),
    ("period_yearly",    pm.Normal.dist(mu=365.0,  sigma=1.00)),
    ("period_weekly",    pm.Normal.dist(mu=7.0,  sigma=0.05)),
    ("ℓ_med",     pm.Gamma.dist(alpha=2,  beta=0.75)), # ls for short-term trend
    ("α",         pm.Gamma.dist(alpha=5,  beta=2)), # shape param for ratquad (short-term)
    ("ℓ_slow",   pm.Gamma.dist(alpha=4,  beta=0.1)), # ls for long-term trend
    ("ℓ_noise",   pm.Gamma.dist(alpha=2,  beta=4))
    ] 

# n = len(priors)
colors = brewer['Paired'][10]

p = figure(title='Lengthscale and period priors',
           plot_width=550, plot_height=350,  x_range=(-1, 21), y_range=(0, 2))
p.yaxis.axis_label = 'Probability'
p.xaxis.axis_label = 'Days'

for i, prior in enumerate(priors):
    p.line(x, np.exp(prior[1].logp(x).eval()), legend_label=prior[0],
           line_width=3, line_color=colors[i])
show(p)

We want the 

In [7]:
x = np.linspace(0, 4, 5000)
priors = [
    ("η_weekly",   pm.HalfCauchy.dist(beta=2)),
    ("η_yearly",   pm.HalfCauchy.dist(beta=1.)),
    ("η_med",   pm.HalfCauchy.dist(beta=1.0)),
    ("η_slow",  pm.HalfCauchy.dist(beta=3)), # will use beta=2, but 2.2 is visible on plot
    ("σ",       pm.HalfNormal.dist(sigma=0.25)),
    ("η_noise", pm.HalfNormal.dist(sigma=0.5))]

colors = brewer['Paired'][10]

p = figure(title='Scale priors',
           plot_width=550, plot_height=350)
p.yaxis.axis_label = 'Probability'
p.xaxis.axis_label = 'Years'

for i, prior in enumerate(priors):
    p.line(x, np.exp(prior[1].logp(x).eval()), legend_label=prior[0],
           line_width=3, line_color=colors[i])
show(p)

In [8]:
# pull out normalized data
t = data_early["t"].values[:,None]
y = data_early["y_n"].values

with pm.Model() as model:
    # yearly periodic component 
    η_per_y = pm.HalfCauchy("η_per_y", beta=1., testval=1.0)
    ℓ_pdecay_y = pm.Gamma("ℓ_pdecay_y", alpha=10, beta=0.075)
    period_y  = pm.Normal("period_y", mu=365., sigma=1.0)
    ℓ_psmooth_y = pm.Gamma("ℓ_psmooth_y", alpha=4, beta=3)
    cov_yearly = η_per_y**2 * pm.gp.cov.Periodic(1, period_y, ℓ_psmooth_y) \
                            * pm.gp.cov.Matern52(1, ℓ_pdecay_y)
    gp_yearly = pm.gp.Marginal(cov_func=cov_yearly)

    # weekly periodic component 
    η_per_w = pm.HalfCauchy("η_per_w", beta=2, testval=1.0)
    ℓ_pdecay_w = pm.Gamma("ℓ_pdecay_w", alpha=10, beta=0.075)
    period_w  = pm.Normal("period_w", mu=7., sigma=0.05)
    ℓ_psmooth_w = pm.Gamma("ℓ_psmooth_w", alpha=4, beta=3)
    cov_weekly = η_per_w**2 * pm.gp.cov.Periodic(1, period_w, ℓ_psmooth_w) \
                            * pm.gp.cov.Matern52(1, ℓ_pdecay_w)
    gp_weekly = pm.gp.Marginal(cov_func=cov_weekly)

    # small/medium term irregularities
    η_med = pm.HalfCauchy("η_med", beta=1., testval=0.1)
    ℓ_med = pm.Gamma("ℓ_med", alpha=2, beta=0.75)
    α = pm.Gamma("α", alpha=5, beta=2)
    cov_medium = η_med**2 * pm.gp.cov.RatQuad(1, ℓ_med, α)
    gp_medium = pm.gp.Marginal(cov_func=cov_medium)

    # long term trend
    η_slow = pm.HalfCauchy("η_slow", beta=3, testval=2.0)
    ℓ_slow = pm.Gamma("ℓ_slow", alpha=4, beta=0.1)
    cov_slow = η_slow**2 * pm.gp.cov.ExpQuad(1, ℓ_slow)
    gp_slow = pm.gp.Marginal(cov_func=cov_slow)

    # noise model
    η_noise = pm.HalfNormal("η_noise", sigma=0.5, testval=0.05)
    ℓ_noise = pm.Gamma("ℓ_noise", alpha=2, beta=4)
    σ  = pm.HalfNormal("σ",  sigma=0.25, testval=0.05)
    cov_noise = η_noise**2 * pm.gp.cov.Matern32(1, ℓ_noise) +\
                pm.gp.cov.WhiteNoise(σ)

    # The Gaussian process is a sum of these three components
    gp = gp_yearly + gp_weekly + gp_medium + gp_slow

    # Since the normal noise model and the GP are conjugates, we use `Marginal` with the `.marginal_likelihood` method
    y_ = gp.marginal_likelihood("y", X=t, y=y, noise=cov_noise)

In [9]:
with model:
    # this line calls an optimizer to find the MAP
    mp = pm.find_MAP(include_transformed=True)

In [ ]:
# display the results, dont show transformed parameter values
sorted([name+":"+str(mp[name]) for name in mp.keys() if not name.endswith("_")])

In [ ]:
# predict at a 15 day granularity
dates = pd.date_range(start='1/01/1969', end="1/01/1976", freq="15D")
tnew = dates_to_idx(dates)[:,None]

print("Predicting with gp ...")
mu, var = gp.predict(tnew, point=mp, diag=True)
mean_pred = mu*std_birth + first_birth
var_pred  = var*std_birth**2

# make dataframe to store fit results
fit = pd.DataFrame({"t": tnew.flatten(),
                    "mu_total": mean_pred,
                    "sd_total": np.sqrt(var_pred)},
                   index=dates)

print("Predicting with gp_slow ...")
mu, var = gp_slow.predict(tnew, point=mp,
                           given={"gp": gp, "X": t, "y": y, "noise": cov_noise},
                           diag=True)
fit = fit.assign(mu_slow = mu*std_birth + first_birth,
                 sd_slow = np.sqrt(var*std_birth**2))

print("Predicting with gp_medium ...")
mu, var = gp_medium.predict(tnew, point=mp,
                            given={"gp": gp, "X": t, "y": y, "noise": cov_noise},
                            diag=True)
fit = fit.assign(mu_med = mu*std_birth + first_birth,
                 sd_med = np.sqrt(var*std_birth**2))

print("Predicting with gp_yearly ...")
mu, var = gp_yearly.predict(tnew, point=mp,
                              given={"gp": gp, "X": t, "y": y, "noise": cov_noise},
                              diag=True)
fit = fit.assign(mu_yearly = mu*std_birth + first_birth,
                 sd_yearly = np.sqrt(var*std_birth**2))

print("Predicting with gp_weekly ...")
mu, var = gp_weekly.predict(tnew, point=mp,
                              given={"gp": gp, "X": t, "y": y, "noise": cov_noise},
                              diag=True)
fit = fit.assign(mu_weekly = mu*std_birth + first_birth,
                 sd_weekly = np.sqrt(var*std_birth**2))

print("Done")

In [ ]:
## plot the components
p = figure(title="Decomposition of the Mauna Loa Data",
           x_axis_type='datetime', plot_width=550, plot_height=350)
p.yaxis.axis_label = 'CO2 [ppm]'
p.xaxis.axis_label = 'Date'

# plot mean and 2σ region of total prediction
upper = fit.mu_total + 2*fit.sd_total
lower = fit.mu_total - 2*fit.sd_total
band_x = np.append(fit.index.values, fit.index.values[::-1])
band_y = np.append(lower, upper[::-1])

# total fit
p.line(fit.index, fit.mu_total,
       line_width=1, line_color="firebrick", legend_label="Total fit")
p.patch(band_x, band_y,
        color="firebrick", alpha=0.6, line_color="white")

# trend
p.line(fit.index, fit.mu_slow,
       line_width=1, line_color="blue", legend_label="Long term trend")

# medium
p.line(fit.index, fit.mu_med,
       line_width=1, line_color="green", legend_label="Medium range variation")

# yearly
p.line(fit.index, fit.mu_yearly,
       line_width=1, line_color="orange", legend_label="Yearly process")

# weekly
p.line(fit.index, fit.mu_weekly,
       line_width=1, line_color="orange", legend_label="Weekly process")

# true value
p.circle(data_early.index, data_early['births'],
         color="black", legend_label="Observed data")
# p.legend.location = "top_left"
show(p)